In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from counterfactuals.datasets import AdultDataset

# from counterfactuals.dequantization import DequantizationOriginal
from counterfactuals.generative_models import MaskedAutoregressiveFlow

In [3]:
dataset = AdultDataset("../data/adult.csv")
dataset.categorical_columns

[2, 3, 4, 5, 6, 7]

In [4]:
quants = [len(cat) for cat in dataset.categorical_features_lists]


gen_model = MaskedAutoregressiveFlow(
    features=dataset.X_test.shape[1],
    hidden_features=4,
    context_features=1,
    category_counts=quants,
    category_indices=dataset.categorical_columns,
)

In [5]:
gen_model.fit(
    train_loader=dataset.train_dataloader(2048, True),
    test_loader=dataset.test_dataloader(2048, False),
    num_epochs=500,
    learning_rate=3e-3,
)

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Epoch 499, Train: 9.3048, test: 9.2609: 100%|██████████| 500/500 [00:58<00:00,  8.54it/s] 


In [6]:
gen_model(
    torch.from_numpy(dataset.X_test[:10]), torch.from_numpy(dataset.y_test[:10])
).exp()

tensor([8.6996e-05, 8.9871e-05, 7.8645e-04, 1.4191e-05, 2.2602e-04, 1.4802e-04,
        4.6260e-06, 2.1161e-03, 5.1729e-06, 2.7269e-04],
       grad_fn=<ExpBackward0>)

In [16]:
gen_model(
    torch.from_numpy(dataset.X_test[:10]), torch.from_numpy(dataset.y_test[:10])
).exp()

tensor([1.6801e-04, 1.9023e+01, 1.1569e+04, 1.0708e+04, 7.6580e+04, 5.5666e+02,
        2.7540e-04, 1.6459e+04, 8.4886e-04, 8.6580e+03],
       grad_fn=<ExpBackward0>)